# Code upgrade objective

Currently, the manner in which high order corrections are applied is supplied as a single option in the input file etc/params.dat. This notebook records the short goal of just adding in the funcationality/option so that different high order corrections (HOC) methods can be applied to different phase space variables. The reason for this upgrade is that the velocity boundary conditions are always periodic, so it makes sense to use the globally accurate Fourier corrections to find the high order corrections which outperform finite differences in accuracy.

The input file etc/params.dat is modified to include a HOC dictionary

    sim_params['HOC'] = {}
    sim_params['HOC']['x'] = 'fd' or 'fourier'
    sim_params['HOC']['vx'] = 'fd' or fourier'
    
and so on. We make a new block similar to the boundary conditions block for the dictionary sim_params['BC'] and permit specification of all HOC corrections for each phase space variable.

Actually, this is kind of entangled. The densities are sent through split routines designated as either all fd or all fourier based corrections. The reason is because the choice of fd vs. fourier determines if phi or E is solved for lib.fieldsolvers (fd is a poission solver, fourier does a Gauss solve). <b>But, the manner in which $E$ is computed depends <i>only</i> on the HOC of the spatial variables.</b> Hence, it depends only on <code>sim_params['HOC']['x'] in our 1D1V case. the value of sim_params['HOC']['vx'] does not enter at this point as a means of selecting which fieldsolver.

To fix this, just create one method called lib.split (as before), then in sim_params store a dictionary HOC, whose key/value pairs are strings HOC['x'] and HOC['vx'] that are either 'fd' or 'fourier'.

Then, inside of lib.split call a general function

     lib.fieldsolvers.compute_electric_field(f, z, vz, sim_params)
     
which then chooses the right path by the dictionary HOC inside, e.g.

    def compute_electric_field(f, z, vz, sim_params):
    
        # if HOC = 'fd'
        
        phi = Poisson_solve(...)
        dphi = W.dot(phi)
        E = -dphi
        
        # if HOC = 'fourier'
        
        E = Gauss(...)
        
But, do this without any if-else checks by selecting with the dictionary strings in HOC.

Could make two functions

    lib.fieldsolvers.compute_electric_field_fd
    
and

    lib.fieldsolvers.compute_electric_field_fourier
    

so that 

    # in lib.split
    
    E = eval(compute_electric_field_function_handle)(f, z, vz, sim_params)
    
chooses the above where

    def compute_electric_field_fd(f, z, vz, sim_params):
        phi = Poisson_6th(...)
        dphi = W.dot(phi)
        E = -dphi
        
        return E
        
and

    def compute_electric_field_fourier(f, z, vz, sim_params):
        E = Gauss(...)
        
        return E
        
alright, done!

